<a href="https://colab.research.google.com/github/spaziochirale/ContemporaryPython/blob/main/Missione8-1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##### Copyright 2019 The TensorFlow Authors.

In [ ]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Word embeddings

In questo tutorial viene introdotto il concetto di embedding delle parole o word embeddings.
Realizzerai un word embeddings utilizzando un semplice modello costruito con Keras per applicazioni di classificazione del "sentiment" e visualizzerai gli embeddings sull'[Embedding Projector](http://projector.tensorflow.org) mostrato nell'immagine qui sotto.

<img src="https://github.com/tensorflow/text/blob/master/docs/tutorials/images/embedding.jpg?raw=1" alt="Screenshot of the embedding projector" width="400"/>

## Rappresentare il testo come numeri

I modelli di machine learning accettano vettori (array di numeri) come input. Quando si lavora con il testo, la prima cosa da fare è ideare una strategia per convertire le stringhe in numeri (o per "vettorializzare" il testo) prima di fornirlo al modello. In questa sezione, esaminerai tre diverse strategie per farlo.

### One-hot encodings

Il one-hot encoding è una tecnica di vettorializzazione del testo che consiste nel rappresentare ogni parola in un vettore binario, dove ogni elemento del vettore corrisponde a una parola del vocabolario. In particolare, il vettore ha lunghezza pari al numero di parole nel vocabolario, e contiene un uno nella posizione corrispondente alla parola da rappresentare, e zeri in tutte le altre posizioni.

Consideriamo come esempio la frase "The cat sat on the mat". Il vocabolario di questa frase è composto dalle seguenti parole: (cat, mat, on, sat, the). Per rappresentare ogni parola utilizzando il one-hot encoding, creiamo un vettore di zeri con lunghezza pari a 5 (il numero di parole nel vocabolario), e posizioniamo un uno nella posizione corrispondente alla parola da rappresentare.
Questo approccio è rappresentato nel diagramma che segue.

<img src="https://www.tensorflow.org/static/text/tutorials/images/one-hot.png" alt="Diagram of one-hot encodings" width="400" />

Per rappresentare una frase utilizzando il one-hot encoding, è possibile concatenare i vettori one-hot di ogni parola nella frase. In questo modo, si ottiene un vettore di lunghezza pari al prodotto tra il numero di parole nella frase e la lunghezza del vettore one-hot di ogni parola.

La concatenazione dei vettori one-hot è un approccio inefficiente per rappresentare le frasi. Ciò è dovuto al fatto che i vettori one-hot sono sparsi, ovvero contengono un numero elevato di zeri. Ad esempio, se il vocabolario contiene 10.000 parole, il vettore one-hot di ogni parola avrà lunghezza 10.000, e conterrà un solo uno e 9.999 zeri. Di conseguenza, la concatenazione dei vettori one-hot di una frase di lunghezza media (ad esempio, 10 parole) produrrà un vettore di lunghezza 100.000, in cui il 99,99% degli elementi sarà zero. Questo approccio è quindi poco efficiente in termini di spazio di memorizzazione e di tempo di elaborazione.

### Codifica di ogni parola mediante un numero unico

Un altro approccio per rappresentare le parole in un modello di machine learning consiste nell'assegnare a ogni parola un numero univoco. In questo modo, ogni parola viene rappresentata da un intero, anziché da un vettore one-hot. Ad esempio, nella frase "The cat sat on the mat", potremmo assegnare il numero 1 alla parola "cat", il numero 2 alla parola "mat", e così via. In questo modo, la frase potrebbe essere rappresentata dal vettore, dove ogni numero corrisponde alla posizione della parola nel vocabolario.

La codifica di ogni parola con un numero univoco è un approccio efficiente per rappresentare le frasi in un modello di machine learning. A differenza del one-hot encoding, che produce vettori sparsi, la codifica di ogni parola con un numero univoco produce vettori densi, in cui tutti gli elementi sono pieni. Ciò significa che il vettore occupa meno spazio di memorizzazione e richiede meno tempo per essere elaborato. Inoltre, la codifica di ogni parola con un numero univoco permette di rappresentare frasi di lunghezza arbitraria, a differenza del one-hot encoding, che richiede una lunghezza fissa del vettore.


Tuttavia, questo approccio presenta ancora due svantaggi.

Il primo svantaggio della codifica di ogni parola con un numero univoco è che tale codifica è arbitraria, ovvero non cattura alcuna relazione tra le parole. Ciò significa che due parole simili dal punto di vista semantico potrebbero essere rappresentate da numeri interi molto diversi, e viceversa. Ad esempio, la parola "gatto" potrebbe essere rappresentata dal numero 1, mentre la parola "cane" potrebbe essere rappresentata dal numero 100. Questa mancanza di relazione tra le parole e i loro numeri interi può rendere difficile per il modello catturare le relazioni semantiche tra le parole.

Il secondo svantaggio è che tale codifica può essere difficile da interpretare per il modello. Ciò è dovuto al fatto che la codifica intera non cattura alcuna relazione tra la similarità di due parole qualsiasi e la similarità delle loro codifiche. Ad esempio, un classificatore lineare apprende un singolo peso per ogni caratteristica del vettore di input. Tuttavia, poiché la codifica intera non cattura alcuna relazione tra le parole, la combinazione di pesi delle caratteristiche non è significativa. Ciò può rendere difficile per il modello apprendere le relazioni semantiche tra le parole e prevedere correttamente l'output desiderato.

### Word embeddings

I word embeddings sono una tecnica di rappresentazione delle parole in uno spazio vettoriale denso e continuo, in cui le parole simili dal punto di vista semantico sono rappresentate da vettori simili. Questa tecnica permette di superare i limiti della codifica one-hot e della codifica intera, che non catturano le relazioni semantiche tra le parole. I word embeddings, invece, catturano queste relazioni in modo efficiente e denso, permettendo al modello di apprendere le relazioni semantiche tra le parole in modo più efficace.

I word embeddings non sono specificati a mano, ma sono appresi dal modello durante l'addestramento come parametri addestrabili. In altre parole, i valori dei vettori di embedding sono pesi appresi dal modello, nello stesso modo in cui un modello apprende i pesi per uno strato denso. Ciò significa che i word embeddings possono essere appresi automaticamente dal modello a partire dai dati di input, senza la necessità di specificare a mano le relazioni semantiche tra le parole.

La dimensionalità dei word embeddings è un parametro che si specifica a priori. È comune vedere word embeddings che sono 8-dimensionali (per dataset piccoli), fino a 1024-dimensioni (per dataset grandi). Una dimensionalità più elevata permette di catturare relazioni fine-grained tra le parole, ma richiede più dati per essere appresa in modo efficace. Al contrario, una dimensionalità più bassa può essere più efficiente dal punto di vista computazionale, ma può catturare meno relazioni semantiche tra le parole. La scelta della dimensionalità dei word embeddings dipende quindi dalla quantità di dati disponibili e dalla complessità del problema da risolvere.


<img src="https://www.tensorflow.org/static/text/tutorials/images/embedding2.png" alt="Diagram of an embedding" width="400"/>

Il diagramma qui sopra rappresenta un word embedding. Ogni parola è rappresentata come un vettore 4-dimensionale di valori in virgola mobile.

I word embeddings possono essere pensati come una "tabella di lookup", in cui ogni parola è associata a un vettore denso di valori in virgola mobile. Dopo che i pesi dei word embeddings sono stati appresi dal modello durante il training, è possibile utilizzare la tabella di lookup per codificare ogni parola nel testo di input come il vettore denso corrispondente. Questo approccio permette di rappresentare le parole in modo efficiente e denso, catturando le relazioni semantiche tra le parole in modo efficace.

Per utilizzare i word embeddings per la codifica delle parole, è necessario prima addestrare il modello su un dataset di testo in input, in modo da apprendere i pesi dei word embeddings.
Dopo che i pesi sono stati appresi, è possibile utilizzare la tabella di lookup per codificare ogni parola nel testo di input come il vettore denso corrispondente. Questo approccio permette di rappresentare le parole in modo efficiente e denso, catturando le relazioni semantiche tra le parole in modo efficace.
Ad esempio, se il modello è stato allenato su un dataset di recensioni di film, le parole "film" e "cinema" potrebbero essere rappresentate da vettori densi simili, riflettendo la loro similarità semantica.


## Setup

In [ ]:
import io
import os
import re
import shutil
import string
import tensorflow as tf

from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense, Embedding, GlobalAveragePooling1D
from tensorflow.keras.layers import TextVectorization

### Download the IMDb Dataset
You will use the [Large Movie Review Dataset](http://ai.stanford.edu/~amaas/data/sentiment/) through the tutorial. You will train a sentiment classifier model on this dataset and in the process learn embeddings from scratch. To read more about loading a dataset from scratch, see the [Loading text tutorial](https://www.tensorflow.org/tutorials/load_data/text).  

Download the dataset using Keras file utility and take a look at the directories.

In [ ]:
url = "https://ai.stanford.edu/~amaas/data/sentiment/aclImdb_v1.tar.gz"

dataset = tf.keras.utils.get_file("aclImdb_v1.tar.gz", url,
                                  untar=True, cache_dir='.',
                                  cache_subdir='')

dataset_dir = os.path.join(os.path.dirname(dataset), 'aclImdb')
os.listdir(dataset_dir)

Take a look at the `train/` directory. It has `pos` and `neg` folders with movie reviews labelled as positive and negative respectively. You will use reviews from `pos` and `neg` folders to train a binary classification model.

In [ ]:
train_dir = os.path.join(dataset_dir, 'train')
os.listdir(train_dir)

The `train` directory also has additional folders which should be removed before creating training dataset.

In [ ]:
remove_dir = os.path.join(train_dir, 'unsup')
shutil.rmtree(remove_dir)

Next, create a `tf.data.Dataset` using `tf.keras.utils.text_dataset_from_directory`. You can read more about using this utility in this [text classification tutorial](https://www.tensorflow.org/tutorials/keras/text_classification).

Use the `train` directory to create both train and validation datasets with a split of 20% for validation.

In [ ]:
batch_size = 1024
seed = 123
train_ds = tf.keras.utils.text_dataset_from_directory(
    'aclImdb/train', batch_size=batch_size, validation_split=0.2,
    subset='training', seed=seed)
val_ds = tf.keras.utils.text_dataset_from_directory(
    'aclImdb/train', batch_size=batch_size, validation_split=0.2,
    subset='validation', seed=seed)

Take a look at a few movie reviews and their labels `(1: positive, 0: negative)` from the train dataset.


In [ ]:
for text_batch, label_batch in train_ds.take(1):
  for i in range(5):
    print(label_batch[i].numpy(), text_batch.numpy()[i])

### Configure the dataset for performance

These are two important methods you should use when loading data to make sure that I/O does not become blocking.

`.cache()` keeps data in memory after it's loaded off disk. This will ensure the dataset does not become a bottleneck while training your model. If your dataset is too large to fit into memory, you can also use this method to create a performant on-disk cache, which is more efficient to read than many small files.

`.prefetch()` overlaps data preprocessing and model execution while training.

You can learn more about both methods, as well as how to cache data to disk in the [data performance guide](https://www.tensorflow.org/guide/data_performance).

In [ ]:
AUTOTUNE = tf.data.AUTOTUNE

train_ds = train_ds.cache().prefetch(buffer_size=AUTOTUNE)
val_ds = val_ds.cache().prefetch(buffer_size=AUTOTUNE)

## Using the Embedding layer

Keras makes it easy to use word embeddings. Take a look at the [Embedding](https://www.tensorflow.org/api_docs/python/tf/keras/layers/Embedding) layer.

The Embedding layer can be understood as a lookup table that maps from integer indices (which stand for specific words) to dense vectors (their embeddings). The dimensionality (or width) of the embedding is a parameter you can experiment with to see what works well for your problem, much in the same way you would experiment with the number of neurons in a Dense layer.


In [ ]:
# Embed a 1,000 word vocabulary into 5 dimensions.
embedding_layer = tf.keras.layers.Embedding(1000, 5)

When you create an Embedding layer, the weights for the embedding are randomly initialized (just like any other layer). During training, they are gradually adjusted via backpropagation. Once trained, the learned word embeddings will roughly encode similarities between words (as they were learned for the specific problem your model is trained on).

If you pass an integer to an embedding layer, the result replaces each integer with the vector from the embedding table:

In [ ]:
result = embedding_layer(tf.constant([1, 2, 3]))
result.numpy()

For text or sequence problems, the Embedding layer takes a 2D tensor of integers, of shape `(samples, sequence_length)`, where each entry is a sequence of integers. It can embed sequences of variable lengths. You could feed into the embedding layer above batches with shapes `(32, 10)` (batch of 32 sequences of length 10) or `(64, 15)` (batch of 64 sequences of length 15).

The returned tensor has one more axis than the input, the embedding vectors are aligned along the new last axis. Pass it a `(2, 3)` input batch and the output is `(2, 3, N)`


In [ ]:
result = embedding_layer(tf.constant([[0, 1, 2], [3, 4, 5]]))
result.shape

When given a batch of sequences as input, an embedding layer returns a 3D floating point tensor, of shape `(samples, sequence_length, embedding_dimensionality)`. To convert from this sequence of variable length to a fixed representation there are a variety of standard approaches. You could use an RNN, Attention, or pooling layer before passing it to a Dense layer. This tutorial uses pooling because it's the simplest. The [Text Classification with an RNN](https://www.tensorflow.org/text/tutorials/text_classification_rnn) tutorial is a good next step.

## Text preprocessing

Next, define the dataset preprocessing steps required for your sentiment classification model. Initialize a TextVectorization layer with the desired parameters to vectorize movie reviews. You can learn more about using this layer in the [Text Classification](https://www.tensorflow.org/tutorials/keras/text_classification) tutorial.

In [ ]:
# Create a custom standardization function to strip HTML break tags '<br />'.
def custom_standardization(input_data):
  lowercase = tf.strings.lower(input_data)
  stripped_html = tf.strings.regex_replace(lowercase, '<br />', ' ')
  return tf.strings.regex_replace(stripped_html,
                                  '[%s]' % re.escape(string.punctuation), '')


# Vocabulary size and number of words in a sequence.
vocab_size = 10000
sequence_length = 100

# Use the text vectorization layer to normalize, split, and map strings to
# integers. Note that the layer uses the custom standardization defined above.
# Set maximum_sequence length as all samples are not of the same length.
vectorize_layer = TextVectorization(
    standardize=custom_standardization,
    max_tokens=vocab_size,
    output_mode='int',
    output_sequence_length=sequence_length)

# Make a text-only dataset (no labels) and call adapt to build the vocabulary.
text_ds = train_ds.map(lambda x, y: x)
vectorize_layer.adapt(text_ds)

## Create a classification model

Use the [Keras Sequential API](https://www.tensorflow.org/guide/keras/sequential_model) to define the sentiment classification model. In this case it is a "Continuous bag of words" style model.
* The [`TextVectorization`](https://www.tensorflow.org/api_docs/python/tf/keras/layers/experimental/preprocessing/TextVectorization) layer transforms strings into vocabulary indices. You have already initialized `vectorize_layer` as a TextVectorization layer and built its vocabulary by calling `adapt` on `text_ds`. Now vectorize_layer can be used as the first layer of your end-to-end classification model, feeding transformed strings into the Embedding layer.
* The [`Embedding`](https://www.tensorflow.org/api_docs/python/tf/keras/layers/Embedding) layer takes the integer-encoded vocabulary and looks up the embedding vector for each word-index. These vectors are learned as the model trains. The vectors add a dimension to the output array. The resulting dimensions are: `(batch, sequence, embedding)`.

* The [`GlobalAveragePooling1D`](https://www.tensorflow.org/api_docs/python/tf/keras/layers/GlobalAveragePooling1D) layer returns a fixed-length output vector for each example by averaging over the sequence dimension. This allows the model to handle input of variable length, in the simplest way possible.

* The fixed-length output vector is piped through a fully-connected ([`Dense`](https://www.tensorflow.org/api_docs/python/tf/keras/layers/Dense)) layer with 16 hidden units.

* The last layer is densely connected with a single output node.

Caution: This model doesn't use masking, so the zero-padding is used as part of the input and hence the padding length may affect the output.  To fix this, see the [masking and padding guide](https://www.tensorflow.org/guide/keras/masking_and_padding).

In [ ]:
embedding_dim=16

model = Sequential([
  vectorize_layer,
  Embedding(vocab_size, embedding_dim, name="embedding"),
  GlobalAveragePooling1D(),
  Dense(16, activation='relu'),
  Dense(1)
])

## Compile and train the model

You will use [TensorBoard](https://www.tensorflow.org/tensorboard) to visualize metrics including loss and accuracy. Create a `tf.keras.callbacks.TensorBoard`.

In [ ]:
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir="logs")

Compile and train the model using the `Adam` optimizer and `BinaryCrossentropy` loss.

In [ ]:
model.compile(optimizer='adam',
              loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
              metrics=['accuracy'])

In [ ]:
model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=15,
    callbacks=[tensorboard_callback])

With this approach the model reaches a validation accuracy of around 78% (note that the model is overfitting since training accuracy is higher).

Note: Your results may be a bit different, depending on how weights were randomly initialized before training the embedding layer.

You can look into the model summary to learn more about each layer of the model.

In [ ]:
model.summary()

Visualize the model metrics in TensorBoard.

In [ ]:
#docs_infra: no_execute
%load_ext tensorboard
%tensorboard --logdir logs

![embeddings_classifier_accuracy.png](https://github.com/tensorflow/text/blob/master/docs/tutorials/images/embeddings_classifier_accuracy.png?raw=1)

## Retrieve the trained word embeddings and save them to disk

Next, retrieve the word embeddings learned during training. The embeddings are weights of the Embedding layer in the model. The weights matrix is of shape `(vocab_size, embedding_dimension)`.

Obtain the weights from the model using `get_layer()` and `get_weights()`. The `get_vocabulary()` function provides the vocabulary to build a metadata file with one token per line.

In [ ]:
weights = model.get_layer('embedding').get_weights()[0]
vocab = vectorize_layer.get_vocabulary()

Write the weights to disk. To use the [Embedding Projector](http://projector.tensorflow.org), you will upload two files in tab separated format: a file of vectors (containing the embedding), and a file of meta data (containing the words).

In [ ]:
out_v = io.open('vectors.tsv', 'w', encoding='utf-8')
out_m = io.open('metadata.tsv', 'w', encoding='utf-8')

for index, word in enumerate(vocab):
  if index == 0:
    continue  # skip 0, it's padding.
  vec = weights[index]
  out_v.write('\t'.join([str(x) for x in vec]) + "\n")
  out_m.write(word + "\n")
out_v.close()
out_m.close()

If you are running this tutorial in [Colaboratory](https://colab.research.google.com), you can use the following snippet to download these files to your local machine (or use the file browser, *View -> Table of contents -> File browser*).

In [ ]:
try:
  from google.colab import files
  files.download('vectors.tsv')
  files.download('metadata.tsv')
except Exception:
  pass

## Visualize the embeddings

To visualize the embeddings, upload them to the embedding projector.

Open the [Embedding Projector](http://projector.tensorflow.org/) (this can also run in a local TensorBoard instance).

* Click on "Load data".

* Upload the two files you created above: `vecs.tsv` and `meta.tsv`.

The embeddings you have trained will now be displayed. You can search for words to find their closest neighbors. For example, try searching for "beautiful". You may see neighbors like "wonderful".

Note: Experimentally, you may be able to produce more interpretable embeddings by using a simpler model. Try deleting the `Dense(16)` layer, retraining the model, and visualizing the embeddings again.

Note: Typically, a much larger dataset is needed to train more interpretable word embeddings. This tutorial uses a small IMDb dataset for the purpose of demonstration.


## Next Steps

This tutorial has shown you how to train and visualize word embeddings from scratch on a small dataset.

* To train word embeddings using Word2Vec algorithm, try the [Word2Vec](https://www.tensorflow.org/tutorials/text/word2vec) tutorial.

* To learn more about advanced text processing, read the [Transformer model for language understanding](https://www.tensorflow.org/text/tutorials/transformer).